[View in Colaboratory](https://colab.research.google.com/github/tcglarry/alibaba_manufacturing_competition/blob/master/ali_xgb_densenet121.ipynb)

In [0]:
"""For Alibaba Competition

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/10DYDNSM_eaFtn7lrM0CyUIXRFdIDtabS
"""

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse






Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls drive/Drive

ls: cannot access 'drive/Drive': No such file or directory


In [0]:

!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

/bin/bash: google-drive-ocamlfuse: command not found


In [0]:
!cp 'drive/My Drive/kaggle/ali/manu_big_data/guangdong_round1_test_a_20180916.zip' .

In [0]:
#!cp drive/kaggle/ali/manu_big_data/train_sample_total.zip .

cp: cannot stat 'drive/kaggle/ali/manu_big_data/train_sample_total.zip': No such file or directory


In [0]:

!unzip guangdong_round1_test_a_20180916.zip


Archive:  train_sample_total.zip
   creating: train_sample_total/
  inflating: train_sample_total/正常20180914144614对照样本_02.jpg  
   creating: __MACOSX/
   creating: __MACOSX/train_sample_total/
  inflating: __MACOSX/train_sample_total/._正常20180914144614对照样本_02.jpg  
  inflating: train_sample_total/横条压凹20180903105700对照样本_02.jpg  
  inflating: __MACOSX/train_sample_total/._横条压凹20180903105700对照样本_02.jpg  
  inflating: train_sample_total/漏底20180912160315对照样本_02.jpg  
  inflating: __MACOSX/train_sample_total/._漏底20180912160315对照样本_02.jpg  
  inflating: train_sample_total/碰伤20180905092240对照样本_02.jpg  
  inflating: __MACOSX/train_sample_total/._碰伤20180905092240对照样本_02.jpg  
  inflating: train_sample_total/正常20180915150646对照样本_02.jpg  
  inflating: __MACOSX/train_sample_total/._正常20180915150646对照样本_02.jpg  
  inflating: train_sample_total/碰伤20180907111235对照样本_02.jpg  
  inflating: __MACOSX/train_sample_total/._碰伤20180907111235对照样本_02.jpg  
  inflating: train_sample_total/正常20180915154327对照样本_02

In [0]:
!ls

adc.json  drive  __MACOSX  MobileNetV2.h5  sample_data	train_sample_total


In [0]:
!pip install --upgrade keras

    100% |████████████████████████████████| 307kB 21.3MB/s 
    100% |████████████████████████████████| 51kB 22.6MB/s 
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6


In [1]:

import keras
print (keras.__version__)
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import subprocess
import os
import pickle
import sklearn 

from sklearn.model_selection import train_test_split

from xgboost.sklearn import XGBClassifier




import keras.backend as K

from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
#from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenetv2 import MobileNetV2,preprocess_input
#from keras.applications.nasnet import NASNetMobile,NASNetLarge
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img



#from keras.applications.xception import Xception, preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout,Cropping2D,Activation
from keras.models import Model, load_model
import pandas as pd

from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import Callback


from keras import regularizers
#from dummyPy import OneHotEncoder
from keras.utils import to_categorical

from sklearn.metrics import confusion_matrix

import itertools
import time

Using TensorFlow backend.


2.2.0


In [2]:
src_1= 'manu_big_data/'
src_2= 'manu_big_data/guangdong_round1_test_a_20180916/guangdong_round1_test_a_20180916/'
src_save = 'manu_big_data/new_era/'

In [3]:
previous_file = pd.read_csv('test_submit_2_st_1.csv',header =None)

previous_file.columns = ['img','label']

test_file_list = previous_file['img'].tolist()

In [4]:
print (len(test_file_list))
print (test_file_list[:10])
with open (src_save+'test_file_list.pkl','wb') as handle:
    pickle.dump(test_file_list,handle)

440
['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']


In [5]:

with open(src_1+'failure_to_label.pkl', 'rb') as handle:
    label_dict = pickle.load(handle)
label_dict  


{'正常': 0,
 '不导': 1,
 '擦花': 2,
 '横条': 3,
 '桔皮': 4,
 '漏底': 5,
 '碰伤': 6,
 '起坑': 7,
 '凸粉': 8,
 '涂层': 9,
 '脏点': 10,
 '其他': 11}

In [6]:
def build_model():
    #inputs = Input(shape=(1920,2560,3))
    inputs = Input(shape=(240,320,3))
   
    # create the base pre-trained model
    #base_model = DenseNet121(weights='imagenet',input_tensor=inputs,  include_top=True)
    base_model = MobileNetV2(weights='imagenet',input_tensor=inputs,  include_top=True)
    #base_model =load_model('drive/My Drive/'+'MobileNetV2.h5')
  
  

 

    model_1 = Model(inputs=inputs, outputs=base_model.output)
    model_1.summary()

    
    
    print ('\n\n*******************************=============================')
    model_2_out = base_model.layers[-3].output
    
    model_2 = Model(inputs=inputs, outputs=model_2_out)
    
    model_2.summary()
    
    return model_2

In [7]:
#model_1, model_2 = build_model()

model_2= build_model()
model_2.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
print ('model compile complete')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 320, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 120, 160, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 120, 160, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (Activation)         (None, 120, 160, 32) 0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
expanded_c

In [8]:
def first_prediction(model,data):
    pred_temp = model.predict(data)
    return pred_temp

In [9]:
def make_batch (file_list,batch_size):
  final_list =[]


  floor = int(np.floor(len(file_list)/batch_size))
  print ('floor=', floor)
  for i in range(floor+1):
      if i < floor:
          final_list.append(file_list[i*batch_size:(i+1)*batch_size])
      else:
          final_list.append(file_list[i*batch_size:])


  # check whther the bach making final_llist len equal to file_list
  a =0
  for l in final_list:
    #print (len(l))
    #print (l[-10:])
    a += len(l)
  if (a - len(final_list) == 0):
    print ('total size', a, 'final_list length equal to file_list')
  else:
    print ('total size', a, 'file_list', len(file_list),'final_list length NOT equal to file_list')
    print (a - len(file_list))
    print ('warning, something wrong')

  return final_list

In [10]:


def make_data(file_list,src):

    
    '''
    first make bacth, then make data
    '''
    batch_size = 7
    final_list = make_batch(file_list,batch_size)
    print (len(final_list))




    s= time.time()
    final_array = []
    for j,l in enumerate(final_list):
          img_list=[]
          for i,img in enumerate(l):
              img = load_img(src_2+img)
              #print (img.size)  

              img = img.resize((2560//8,1920//8))
        

              x = img_to_array(img).astype('uint')
              my_filter = (x.max(axis=2) - x.min(axis=2)) < 50  # get mask 

              for k in range(3):

                    x[:,:,k] *= my_filter  # mask out the unnecessary part  


              #print ('0',x.shape)  
              x = x/255.

              if i  == 0 :


                print ('1', x.shape)
              #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
              #x = x/127.5 -1.
              img_list.append(x)
          #print ('img_list=', len(img_list))
          testing_data = np.array(img_list)
          pred_temp = first_prediction(model_2,testing_data)  

          #print ('testing_data_shape', testing_data.shape)
          #train_data =  train_data[:,:,:,np.newaxis]
          #print (testing_data.shape)
          
          e = time.time()
          print ('batch',j,'time=', round(e -s ,4))

          final_array.append(pred_temp)
          print ('afterbatch',j,'length=', len(final_array))

    final_array = np.concatenate(final_array)

    return final_array

In [11]:
test_final_array = make_data(test_file_list, src_2)

print ('test final array', test_final_array.shape)
np.save(src_save+'test_data_aug_small_for_2_gap.npy',test_final_array)
print ('totally done, test_data_aug_small_for_2_gap.npy')



floor= 62
total size 440 file_list 440 final_list length NOT equal to file_list
0
warning, something wrong
63
1 (240, 320, 3)
batch 0 time= 4.8837
afterbatch 0 length= 1
1 (240, 320, 3)
batch 1 time= 8.522
afterbatch 1 length= 2
1 (240, 320, 3)
batch 2 time= 12.1171
afterbatch 2 length= 3
1 (240, 320, 3)
batch 3 time= 15.9304
afterbatch 3 length= 4
1 (240, 320, 3)
batch 4 time= 20.1189
afterbatch 4 length= 5
1 (240, 320, 3)
batch 5 time= 24.4346
afterbatch 5 length= 6
1 (240, 320, 3)
batch 6 time= 28.8561
afterbatch 6 length= 7
1 (240, 320, 3)
batch 7 time= 33.5763
afterbatch 7 length= 8
1 (240, 320, 3)
batch 8 time= 38.4313
afterbatch 8 length= 9
1 (240, 320, 3)
batch 9 time= 43.273
afterbatch 9 length= 10
1 (240, 320, 3)
batch 10 time= 47.2663
afterbatch 10 length= 11
1 (240, 320, 3)
batch 11 time= 51.4066
afterbatch 11 length= 12
1 (240, 320, 3)
batch 12 time= 57.7618
afterbatch 12 length= 13
1 (240, 320, 3)
batch 13 time= 63.8952
afterbatch 13 length= 14
1 (240, 320, 3)
batch 14 ti